In [5]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from io import BytesIO
from urllib.parse import urljoin, urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from PIL import Image
import os
import random
import re
import pandas as pd
import hashlib
from datetime import datetime
import urllib.parse
from json import dumps, loads
from shutil import copy2

In [7]:
url = 'https://www.dailymail.co.uk/sport/index.html'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')
page_container = soup.find('div', id='page-container')
info_headers = page_container.select('div[itemscope][itemtype]')

In [138]:
url = 'https://www.dailymail.co.uk/sport/index.html'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')
page_container = soup.find('div', id='page-container')
info_headers = page_container.select('div[itemscope][itemtype]')
articles_links = []

for news_card in info_headers:
    a_tag = news_card.find('a', attrs={'itemprop': 'url'})
    link = f"https://www.dailymail.co.uk{a_tag['href']}"
    if link not in articles_links:
        articles_links.append(link)

In [139]:
articles_links

['https://www.dailymail.co.uk/sport/tennis/article-13820593/Jack-Draper-vs-Jannik-Sinner-Open-LIVE-Resuly.html',
 'https://www.dailymail.co.uk/sport/football/article-13822245/Sol-Campbell-KNIGHTHOOD-England.html',
 'https://www.dailymail.co.uk/sport/football/article-13820413/James-Maddison-England-Lee-Carsley-Tottenham.html',
 'https://www.dailymail.co.uk/sport/football/article-13822909/Chelseas-majority-owners-Clearlake-Capital-not-selling.html',
 'https://www.dailymail.co.uk/sport/skybet/article-13758215/Trent-Alexander-Arnold-England-Nations-League-Ireland.html',
 'https://www.dailymail.co.uk/sport/football/article-13822533/Chelsea-dressing-room-difficult-Enzo-Marescas-Marc-Cucurella.html',
 'https://www.dailymail.co.uk/sport/football/article-13816675/Evertons-maximum-impact-dressing-room-rollicking-left-no-place-hide-amid-growing-fears-crisis-hit-club-sold.html',
 'https://www.dailymail.co.uk/sport/football/article-13822391/League-One-coach-join-Real-Madrid-backroom-staff-Carlo-Anc

In [107]:
resp = requests.get('https://www.dailymail.co.uk/sport/football/article-13822481/womens-championship-football-season-financial.html')
soup = BeautifulSoup(resp.content, 'lxml')

In [110]:
time_tag = soup.find('time')

In [116]:
# datetime.fromisoformat(time_tag['datetime'])
datetime_str = time_tag['datetime']
datetime_obj = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S%z")

print(datetime_obj) 

2024-09-06 18:47:26+01:00


In [105]:
images = []
ads = []

for tag in div_tag.find_all('div'):
    img = tag.find('img')
    class_name = tag.get('class', [])
    
    if img:
        img_alt = img.get('alt', '')  # Safely get the 'alt' attribute
        img_src = img.get('src', '')  # Safely get the 'src' attribute
        
        if img_src and (img_alt, img_src) not in images:
            images.append((img_alt, img_src))

        if class_name in [['floatRHS'], ['moduleFull']]:
            ads.append((img_alt, img_src))

set(images) - set(ads)


In [106]:
set(images) - set(rem)

{("Being tied to a Premier League men's side such as Man United brings better financial rewards",
  'https://i.dailymail.co.uk/1s/2024/09/06/18/89369947-0-image-a-11_1725643319400.jpg'),
 ('Bristol City\xa0are set to start the season without a manager after Lauren Smith (pictured) left the club earlier this month',
  'https://i.dailymail.co.uk/1s/2024/09/06/18/89369355-0-image-a-3_1725642588655.jpg'),
 ('Championship\xa0clubs have previously had to submit their salary cap figures twice a year',
  'https://i.dailymail.co.uk/1s/2024/09/06/18/89369737-0-image-a-10_1725643011530.jpg'),
 ('Crystal Palace have been promoted to the WSL after winning the Championship last season',
  'https://i.dailymail.co.uk/1s/2024/09/06/18/89370259-0-image-a-14_1725644252942.jpg'),
 ("It also remains to be seen what impact Amanda Staveley¿s (pictured) departure will have on Newcastle's women's team",
  'https://i.dailymail.co.uk/1s/2024/09/06/18/89369959-0-image-a-12_1725643438916.jpg'),
 ('London City Lion

In [120]:
def create_directories(base_url, categories, label='images'):
    # create the following dir struct; outputs > base website > categories
    base_dir = os.path.join('output', label, urlparse(base_url).netloc)
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
    
    for category, _ in categories:
        category_dir = os.path.join(base_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

    return base_dir

In [143]:
def get_articles_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    page_container = soup.find('div', id='page-container')
    info_headers = page_container.select('div[itemscope][itemtype]')
    articles_links = []

    for news_card in info_headers:
        a_tag = news_card.find('a', attrs={'itemprop': 'url'})
        link = f"https://www.dailymail.co.uk{a_tag['href']}"
        if link not in articles_links:
            articles_links.append(link)
    
    return articles_links

def article_scrapper(url):
    articles_links = get_articles_links(url)
    data = []

    def helper_scrapper(url):   
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'lxml')
        try:
            time_tag = soup.find('time')
            datetime_str = time_tag['datetime']
            time = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S%z")
            headline = soup.find('h1').text
            div_tag = soup.find('div', id='js-article-text', class_='article-text wide heading-tag-switch')

            images = []
            ads = []

            for tag in div_tag.find_all('div'):
                img = tag.find('img')
                class_name = tag.get('class', [])
                
                if img:
                    img_alt = img.get('alt', '')  # Safely get the 'alt' attribute
                    img_src = img.get('src', '')  # Safely get the 'src' attribute
                    
                    if img_src and (img_alt, img_src) not in images:
                        images.append((img_alt, img_src))

                    if class_name in [['floatRHS'], ['moduleFull']]:
                        ads.append((img_alt, img_src))

            return headline, time, list(set(images) - set(ads))
        except:
            return None
        
    for url in articles_links:   
        result = helper_scrapper(url)
        if result is not None:
            data.append(result)
    
    return data

In [149]:
def get_latest_articles(data, n=10):
    seen_headlines = set()
    unique_data = []

    for record in data:
        headline = record[0]
        if headline not in seen_headlines:
            seen_headlines.add(headline)
            unique_data.append(record)

    return sorted(unique_data, key=lambda x: x[1], reverse=True)[:n]

def download_image(img_url, save_dir, img_name):
    try:
        if not img_url.startswith('data:'):
            response = requests.get(img_url)
            img_data = response.content
            img = Image.open(BytesIO(img_data))
            width, height = img.size

            # Only save images larger than 100x100 pixels
            if width >= 100 and height >= 100:
                with open(os.path.join(save_dir, img_name), 'wb') as img_file:
                    img_file.write(img_data)
    except:
        pass

def download_images(category_url, save_dir, data):

    with open(os.path.join(save_dir, 'labels.csv'), 'w') as f:
        f.write('image number,alt,article_heading\n')
    
    records = []

    # parallising the downloads to make it faster
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        headlines = []
        for x, tuple in enumerate(data):
            headline, _, images_list = tuple
            for i, img in enumerate(images_list):
                alt_txt, img_url = img
                if img_url and not img_url.startswith('data:'):
                    img_url = urljoin(category_url, img_url)
                    combined_str = f"{alt_txt}{headline}".encode()
                    img_name = f'image_{x+1}{i+1}.jpg'
                    records.append(f'{img_name},{alt_txt.replace(",", "")},{headline.replace(",", "")}\n')
                    futures.append(executor.submit(download_image, img_url, save_dir, img_name))
        
        with open(os.path.join(save_dir, 'labels.csv'), 'a') as f:
            f.writelines(records)
            
        for future in as_completed(futures):
            future.result()


In [150]:
categories = [
    ("U.S.", "https://www.dailymail.co.uk/ushome/index.html"),
    ("Sport", "https://www.dailymail.co.uk/sport/index.html"),
    ("TV", "https://www.dailymail.co.uk/tv/index.html"),
    ("Showbiz", "https://www.dailymail.co.uk/tvshowbiz/index.html"),
    ("Femail", "https://www.dailymail.co.uk/femail/index.html"),
    ("Health", "https://www.dailymail.co.uk/health/index.html"),
    ("Science", "https://www.dailymail.co.uk/sciencetech/index.html"),
    ("Money", "https://www.dailymail.co.uk/money/index.html"),
    ("Travel", "https://www.dailymail.co.uk/travel/index.html"),
    ("Shopping", "https://www.dailymail.co.uk/shopping-uk/index.html"),
    ("Royals", "https://www.dailymail.co.uk/news/royals/index.html"),
    ("Australia", "https://www.dailymail.co.uk/auhome/index.html"),
    ("You Mag", "https://www.dailymail.co.uk/home/you/index.html"),
    ("Books", "https://www.dailymail.co.uk/home/books/index.html"),
    ("Cars", "https://www.dailymail.co.uk/money/cars/index.html"),
    ("Property", "https://www.dailymail.co.uk/property/index.html"),
]

base_url = 'https://www.dailymail.co.uk/'

base_dir = create_directories(base_url, categories)

In [151]:
for category, category_url in tqdm(categories, desc='Downloading images for every category'):
    try:
        category_dir = os.path.join(base_dir, category)
        data = article_scrapper(category_url)
        download_images(category_url, category_dir, get_latest_articles(data))
    except:
        continue

In [2]:
def create_file(base_dir):
    for category in os.listdir(base_dir):
        df = pd.read_csv(f'{base_dir}{category}/labels.csv')
        n, _ = df.shape

        pairs = []

        for i in range(n):
            for j in range(i + 1, n):
                pairs.append((list(df.iloc[i]), list(df.iloc[j])))

        with open(f'{base_dir.split("/")[2]}_pairs_{category}.csv', 'w') as f:
            for i, pair in enumerate(pairs):
                p1, p2 = pair
                img1, alt1, headline1 = p1
                img2, alt2, headline2 = p2
                f.write(f'{i+1},{headline1},{headline2}\n')
                f.write(f',https://raw.githubusercontent.com/ayainfida/news-scrapper/main/output/images/www.dailymail.co.uk/{urllib.parse.quote(category)}/{img1},https://raw.githubusercontent.com/ayainfida/news-scrapper/main/output/images/www.dailymai/{urllib.parse.quote(category)}/{img2}\n')
                f.write(f',{alt1},{alt2}\n')

In [6]:
create_file('output/images/www.dailymail.co.uk/')